In [1]:
import gradio as gr
import google.generativeai as genai
from typing import List, Tuple
from config import GEMINI_API_KEY, MEDICAL_SYSTEM_PROMPT

b:\LLM_Medical_Project\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Available names in config: ['DB_CONFIG', 'GEMINI_API_KEY', 'MEDICAL_SYSTEM_PROMPT', 'OPENAI_API_KEY', 'OPENFDA_API_KEY', 'OPENFDA_API_URL', 'PUBMED_API_KEY', 'PUBMED_API_URL', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'config']


In [2]:
class GeneralConsultation:
    def __init__(self):
        """일반 상담 모듈 초기화"""
        genai.configure(api_key=GEMINI_API_KEY)
        self.model = genai.GenerativeModel('gemini-pro')
        self.system_prompt = MEDICAL_SYSTEM_PROMPT
    
    def get_medical_response(self, message: str, history) -> str:
        """의료 상담 응답 생성"""
        chat = self.model.start_chat()
        full_prompt = f"{self.system_prompt}\n\n사용자: {message}"
        response = chat.send_message(full_prompt)
        return response.text
    
    def create_interface(self) -> None:
        """Gradio 인터페이스 생성"""
        demo = gr.ChatInterface(
            fn=self.get_medical_response,
            title="의료 상담 챗봇",
            description="의료 관련 질문에 대해 답변해드립니다. (전문의와 상담을 대체할 수 없습니다)",
            examples=[
                ["당뇨병의 초기 증상은 무엇인가요?"],
                ["고혈압을 예방하기 위한 생활 수칙을 알려주세요."],
                ["편두통과 일반 두통의 차이점은 무엇인가요?"]
            ]
        )
        return demo

In [3]:
if __name__ == "__main__":
    consultation = GeneralConsultation()
    demo = consultation.create_interface()
    demo.launch()

b:\LLM_Medical_Project\.venv\lib\site-packages\gradio\components\chatbot.py:279: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


In [4]:
demo.close()

Closing server running on port: 7861
